In [1]:
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import HTML

ModuleNotFoundError: No module named 'altair'

# 上市公司利润分析

## 数据准备
- 从交易所取下的上市公司列表，不包括B股。
- 从东方财富取下的行业分类数据。
- 从东方财富取下的业绩快报数据。

In [7]:
def showProfile(df) :
    col_profile = pd.concat([df.dtypes, df.isna().sum()/len(df)*100, df.sample().T], axis=1 ).reset_index()
    col_profile.columns = ['列名', '类型', '缺失比%','样例']
    print(df.shape)
    return (col_profile)

In [8]:
# load data
code = pd.read_pickle('../../ex_list_basic_20230529.pkl')
indu = pd.read_pickle('../../em_code_indu.pkl')
df = pd.read_pickle('../../yjbg2022q4.pkl.gz')

In [9]:
showProfile(df)

(320679, 11)


,列名,类型,缺失比%,样例
0,股票代码,object,0.000000,872810
1,股票简称,object,0.000000,国拓科技
2,每股收益,float64,1.772489,0.22
3,营业收入-营业收入,float64,0.188974,57384790.22
4,净利润-净利润,float64,0.026506,10822362.5
5,每股净资产,float64,4.812289,2.842418
6,净资产收益率,float64,2.447307,7.92
7,每股经营现金流量,float64,4.056393,0.311391
8,销售毛利率,float64,2.401467,43.542055
9,最新公告日期,object,0.000000,2022-10-25


In [10]:
# select A share from all codes
code = code[code.market != '主板B股']

code['ly'] = pd.to_datetime(code['list_day']).dt.year
code['ly'] = code['ly'].apply(lambda x: x if x>2010 else 2010)
# exclude 2023 ipos
code_sel = code[code['ly'] < 2023]

# select yearly profit from yjbg
df = df.iloc[:,[0,1,4,10]]
df.columns = ['code', 'name', 'profit','year']
sel = df[(df['year'].str[4:] == '1231') & (df.code.isin(code_sel.code)==True)]
sel.loc[:,'year'] = sel['year'].str[0:4].astype('int32')
sel = sel[sel['year'] > 2009]

# merge listed year to the profit df
sel = sel.merge(code_sel[['code','ly']])
# select data only in and after list year
sel = sel[sel['year'] >= sel['ly']]

In [11]:
showProfile(sel)

(41283, 5)


,列名,类型,缺失比%,样例
0,code,object,0.0,002522
1,name,object,0.0,浙江众成
2,profit,float64,0.0,64161660.67
3,year,object,0.0,2019
4,ly,int64,0.0,2010


In [12]:
# pivot the sum of each year
sel_table = sel.pivot_table(index='ly', columns='year', values='profit', aggfunc='sum')

# 分别计算出总利润，总利润增长，新上市引起的利润增长，以及已上市公司的利润增长。

# 总利润
total_sum= sel_table.sum()
# 总利润增长
total_inc = total_sum.diff()
total_inc.values[0] = 0
# 新上市引起的利润增长
li = []
for i in range(0, len(sel_table)):
    li.append(sel_table.iat[i,i])
new_inc = pd.Series(li, index=total_inc.index)
new_inc.values[0] = 0
# 已上市公司的利润增长
old_inc = total_inc - new_inc
old_inc.values[0] = 0
# 上一年基数
base = total_sum - new_inc - old_inc
base.values[0] = total_sum.values[0]

inc_df = pd.DataFrame([base, old_inc, new_inc]).T
inc_df.columns = ['base', '增长驱动', '新发驱动']

In [13]:
# 利润的一些统计值
profit_quant = sel.groupby('year')['profit'].agg(['mean', 'median', 'idxmin', 'idxmax', 'count']).reset_index()


## 上市公司利润总和的变化
- 上市当年的数据计入上市公司总盘子。上市之前的数据不计入。
- 如果把全体上市公司当做一个公司，以2010年底数为基数，则近12年来的几何平均增长率为10%

In [14]:
df_total = total_sum.to_frame('profit').reset_index()
alt.Chart(df_total).mark_bar().encode(
    alt.X('profit:Q').axis(format='.2s'),
    alt.Y('year:O')
).properties(
    title = '总利润变化',
    width = 'container',
    height = 340
)

alt.Chart(...)

In [15]:
# card the average increace persent
a= pow((total_sum.values[-1]/total_sum.values[0]) , (1/12))-1
avi = f'{a:.2%}'
sty = '''
  <style>
      .card_group {
        display: flex;
        flex-wrap: wrap;
        justify-content: center;
        text-align: center;
      }

      .card {
        display: flex;
        flex-direction: column;
        justify-content: center;
        align-items: center;
        background: #fff;
        border-radius: 10px;
        display: inline-block;
        height: 260px;
        margin: 1rem;
        width: 260px;
        box-shadow: 0 3px 6px rgba(0, 0, 0, 0.12), 0 3px 6px rgba(0, 0, 0, 0.24);
      }

      .card_title {
        border-bottom: 5px;
        border-bottom-style: dotted;
        border-bottom-color: grey;
        margin: 5px;
        padding: 5px;
        font-size: 2em;
      }
      .card_content {
        font-size: 4em;
        margin: 5px;
        align-self: center;
        height: 200px;

        display: flex;
        align-items: center;
        justify-content: center;
        justify-items: center;
      }
    </style>
'''

card = f'''
      {sty}
      <div class ='card_group'>
      <div class = 'card'>
      <div class = 'card_title'>总体年增长率</div>
      <div class = 'card_content'>{avi}</div>
      </div>
      </div>
'''
HTML(card)

- 上市公司总体利润保持增长,特别是疫情爆发后第2年,即2021年,出现了爆涨.
- 近12年的增长率为10%

以下观察利润的主要统计指标:

In [16]:
alt.Chart(profit_quant).mark_bar().encode(
    alt.X('mean:Q').axis(format='.2s'),
    alt.Y('year:O')
).properties(
    title = '上市公司利润平均值变化',
    width = 'container',
    height = 340
)

alt.Chart(...)

In [17]:
alt.Chart(profit_quant).mark_bar().encode(
    alt.X('median:Q').axis(format='.2s'),
    alt.Y('year:O')
).properties(
    title = '上市公司利润中位数变化',
    width = 'container',
    height = 340
)

alt.Chart(...)

- 上市公司的平均利润总体略有增长，但2022年有较大下降，大约为10亿左右。
- 上市公司利润中位数在2022年急剧下降，略高于1亿。
- 平均值和中位数差一个数量级，说明上市公司利润的分布是左偏的。

In [18]:
mindf = sel.loc[profit_quant['idxmin'],['year','code', 'name', 'profit']].reset_index(drop = True)
base = alt.Chart(mindf).encode(
    alt.X('profit:Q').axis(format='.2s'),
    alt.Y('year:O'),
    text='name'
).properties(
    title = '历年亏损王',
    width = 'container',
    height = 340
)
base.mark_bar(color='brown') + base.mark_text(align='left', color='white')


alt.LayerChart(...)

- 上市公司亏损王呈现出不断创新高的情况。
- 谁又能想到中海远控，曾经勇巨亏王呢？
- 疫情来临之际，中海远控大翻身，而同为交通行业的大航空，却整整苦了三年。
- 看到其中的一个亏损王，感觉自己也算是亲历一段历史。

In [19]:
maxdf = sel.loc[profit_quant['idxmax'],['year','code', 'name', 'profit']].reset_index(drop = True)
base = alt.Chart(maxdf).encode(
    alt.X('profit:Q').axis(format='.2s'),
    alt.Y('year:O'),
    text='name'
).properties(
    title = '历年赢利王',
    width = 'container',
    height = 340
)
base.mark_bar() + base.mark_text(align='right', color='white')

alt.LayerChart(...)

- 赢利王没什么惊喜，宇宙第一行，永远行，疫情完全没受到影响。可怕。

## 利润构成-利润增长和新增上市公司驱动
- 上市公司上市当年的利润记为新增上市公司驱动的利润，象是给整个池子注入资本。
- 上市公司上市后第二年的利润增长做为池子本身的利润增长驱动。
- 上市公司的利润增长是不是IPO的因素更重要呢？

In [20]:
alt.Chart(profit_quant).mark_bar().encode(
    alt.X('count:Q').axis(format='.0f'),
    alt.Y('year:O')
).properties(
    title = '上市公司报告数量变化',
    width = 'container',
    height = 340
)

alt.Chart(...)

- 最近几年IPO增长很快，利润增长是不是IPO驱动的呢？

In [21]:
inc_plotdf = inc_df.reset_index().melt(id_vars='year', var_name='cat',value_name='profit')
alt.Chart(inc_plotdf).mark_bar().encode(
    alt.X('sum(profit)').axis(format='.2s'),
    alt.Y('year:O'),
    color='cat'
).properties(
    title = '利润变化驱动因素',
    width = 'container',
    height = len(df_total)*20+80
)

alt.Chart(...)

- 可以看出，大部分时候，是已上市公司利润增长驱动的。只有少数例外。
- 其中，2015年和最近2022年，出出了已上市公司亏损，利润增长全部由当年新发公司驱动。
- 从大规律上看，上市公司的利润在爆发后，容易第二年就疲软。2017年和2021年就是个例子。
- 下图上两个驱动因素的详细对比。

In [22]:
inc_plotdf_sel = inc_plotdf[inc_plotdf['cat'] != 'base']
alt.Chart(inc_plotdf_sel).mark_bar().encode(
    alt.X('sum(profit)').axis(format='.2s'),
    alt.Y('year:O'),
    color='cat'
).properties(
    title = '两种利润增长驱动因素对比',
    width = 'container',
    height = len(df_total)*20+80
)


alt.Chart(...)

- 谁能想到疫情第2年，上市公司却出现史诗性的暴涨。
- 而到了第三年，增长不再还转为亏损。

## 利润构成——按板块和交易所

- 这方面应该符合常识认知。

In [23]:
sel_e_m = sel.merge(code[['code', 'market', 'ex']])
sel_bymarket  =sel_e_m.groupby(['year','market'])['profit'].sum().to_frame('profit').reset_index()
alt.Chart(sel_bymarket).mark_bar().encode(
    alt.X('sum(profit)').axis(format='.2s'),
    alt.Y('year:O'),
    color='market'
).properties(
    title = '不同板块利润变化',
    width = 'container',
    height = 340
)


alt.Chart(...)

In [24]:
sel_byex  =sel_e_m.groupby(['year','ex'])['profit'].sum().to_frame('profit').reset_index()
alt.Chart(sel_byex).mark_bar().encode(
    alt.X('sum(profit)').axis(format='.2s'),
    alt.Y('year:O'),
    color='ex'
).properties(
    title = '不同交易所利润变化',
    width = 'container',
    height = 340
)


alt.Chart(...)

- 2020年对上交所打击比较大。
- 但第二年2021年，上交所又迎来狂欢。

## 利润构成-分行业看

- 一般人可能不知道，中国上市银行有多挣钱，不然无法解释为什么银行股在大部分时候都被嫌弃。
- 先看一下从2010年以来，每年的利润额前十榜上有哪些公司。

In [25]:
# 先按年份和利润额排列，然后取每年的前10名。
top10 = sel.sort_values(by=['year', 'profit'], ascending=[True, False]).groupby('year', as_index=False).nth[:10]['name'].value_counts().to_frame('count').reset_index()

alt.Chart(top10).mark_bar().encode(
    alt.X('count:Q'),
    alt.Y('name:O').sort('-x'),
).properties(
    title = '2010至2022利润额前十上榜次数统计',
    width = 'container',
    height = 440
)

alt.Chart(...)

- 每年都上榜的有6家，全是银行。而且，工商银行每年都是冠军。
- 上榜的共有19家公司，银行占了11家，如果把平安和人寿算进去，金融占了13家。
- 4家资源性公司，三桶油加一个煤炭。
- 1家交通运输企业，中远海控。应该就是2021年疫情上的榜。世事无常，它还曾夺过亏损冠军。
- 1家高科技企业，中国移动。

In [26]:
indu_counts = indu.groupby('indu')['code'].count().sort_values().to_frame('counts').reset_index()
alt.Chart(indu_counts).mark_bar().encode(
    alt.Y('counts:Q').bin(step = 20),
    alt.X('count()')
).properties(
    width='container',
    height=400
)

alt.Chart(...)